# Training and evaluation of modified IM2ELEVATION model

In [1]:
# Ensures runtime code is updated when source code of libraries are updated as well
%load_ext autoreload
%autoreload 2

In [2]:
# Import of necessary libraries

import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import numpy as np
import pickle
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import ssl

from osgeo import gdal
import geopandas as gpd

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# FastAI
from fastai.vision.all import *
from fastai.text.all import *
from fastai.collab import *
from fastai.tabular.all import *

import utils.sp_utils as sp


In [3]:
# User modules

from utils.models import senet
from utils.models import modules
from utils.models import net

In [4]:
# Check if gpu/cuda is available
import torch
torch.cuda.is_available()

True

## Build Dataset

In [34]:
class UMPDataset(Dataset):
    """Urban Morphological Parameters - Sentinel Dataset"""

    def __init__(self, ump_df, dir_path, file_ext= ".tiff", transform= None):
        """
        # Parameters:
            - ump_df: DataFrame containing the geometry of each cell with its corresponding UMPs.
            - dir_path: Directory in which all tiff files will be matched with it's corresponding cells in ump_df.
                - No trailing slashes for dir. eg. "data/osaka"
                - Naming convention for files: Sentinel_{'_'.join(map(str, map(int, cell.bounds)))}.tiff for cell in ump_df["geometry"]
            - file_ext: Extension of the file eg. .tiff / .tif
        """
        self.ump_df = ump_df
        self.dir_path = dir_path
        self.file_ext = file_ext
        # Look in both current and sub-directories
        self.files = glob.glob(f"{self.dir_path}/**/*{self.file_ext}") + glob.glob(f"{self.dir_path}/*{self.file_ext}")
        self.transform = transform
    
    def __len__(self):
        return len(self.ump_df)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        poly = self.ump_df["geometry"][idx]
        target = f"Sentinel_{'_'.join(map(str, map(int, poly.bounds)))}.tiff"
        # Finds and return the first match in a string
        file_name = next((s for s in self.files if target in s), None)
        if file_name is None:
            raise KeyError(f"{target} not found in {self.dir_path}")
        
        image = gdal.Open(file_name, gdal.GA_ReadOnly).ReadAsArray()
        umps = list(Y_osaka.iloc[idx][[
            "AverageHeightArea", 
            "AverageHeightBuilding", 
            "AverageHeightTotalArea", 
            "Displacement", 
            "FrontalAreaIndex",
            "MaximumHeight",
            "PercentileHeight",
            "PlanarAreaIndex",
            "RoughnessLength",
            "StandardDeviation",
        ]])

        out = {"image":image, "umps":umps}

        if self.transform:
            out = self.transform(out)
        
        return out
        



In [35]:
Y_osaka = gpd.read_feather("data/Y_UMP/Y_osaka_3.feather")
ds_osaka = UMPDataset(Y_osaka, "data/X_sentinel/osaka")

In [30]:
glob.glob("data/X_sentinel/osaka/**/*.tiff")

[]

In [37]:
ds_osaka[0]["image"].shape

(12, 90, 90)

In [4]:
files = ["aa", "aaa", "bbb", "aaa"]
next((s for s in files if "a" in s), None)

'aa'

In [13]:
Y_osaka = gpd.read_feather("data/Y_UMP/Y_osaka_3.feather")
Y_osaka[pd.DataFrame.any(Y_osaka.isna(), axis= 1)]
# Y_osaka

,geometry,AverageHeightArea,AverageHeightBuilding,AverageHeightTotalArea,Displacement,FrontalAreaIndex,MaximumHeight,Percentile,PercentileHeight,PlanarAreaIndex,RoughnessLength,StandardDeviation


In [24]:
list(Y_osaka.iloc[0][[
    "AverageHeightArea", 
    "AverageHeightBuilding", 
    "AverageHeightTotalArea", 
    "Displacement", 
    "FrontalAreaIndex",
    "MaximumHeight",
    "PercentileHeight",
    "PlanarAreaIndex",
    "RoughnessLength",
    "StandardDeviation",
]])

[23.321947080728076,
 9.481818181818182,
 0.2878379437277183,
 0.0009568369634676615,
 0.005289593160466849,
 38.4,
 33.90000000000001,
 0.012341934519076716,
 0.00022359466733766674,
 3.0691962418986156]